# Analisi e Implementazione di Metodi di Ottimizzazione

## 1 Import delle librerie

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use("TkAgg")
from mpl_toolkits.mplot3d import Axes3D # Per grafici 3D

# 2 Implementazione degli Algoritmi

## 2.1 Funzione Backtracking (Line Search)

In [2]:
def backtracking(f, df, X_k, p_k):
    alpha = 1
    rho = 1/2
    c1 = 0.25
    grad_k = df(X_k)
    grad_dot_p = np.dot(grad_k, p_k)    
    # Condizione Di Armijo
    while f(X_k + alpha * p_k) > f(X_k) + c1 * alpha * grad_dot_p:
        alpha = rho * alpha
    return alpha

## 2.2 Metodo del gradiente

In [3]:
def GD (f, df, X_old, maxit, tol_f, tol_x):
    dim = len(X_old)
    fun_history = np.zeros((maxit + 1, dim))
    fun_history[0,] = X_old
    exit_flag = 'maxit'

    current_grad_norm=np.linalg.norm(df(X_old))

    for count in range (maxit):
        if current_grad_norm < tol_f:
            break
        p_k=-df(X_old)
        alpha=backtracking(f, df, X_old, p_k)
        X_new = X_old + alpha * p_k
        if np.linalg.norm(X_new-X_old) < tol_x:
            exit_flag = 'tol_x'
            break
        current_grad_norm=np.linalg.norm(df(X_new))
        X_old = X_new
        count+=1
        fun_history[count] = X_new

    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    if count<maxit:
        fun_history = fun_history[:count+1]

    return X_old, count, fun_history, exit_flag

## 2.2 Metodo Newton

In [4]:
def Newton (f, df, hess_f, X_old, maxit, tol_f, tol_x):
    dim = len(X_old) 
    fun_history = np.zeros((maxit + 1, dim))
    fun_history[0,]=X_old
    exit_flag = 'maxit'

    current_grad_norm=np.linalg.norm(df(X_old))

    for count in range(maxit):

        if current_grad_norm < tol_f:
            break

        Hess_k = hess_f(X_old)

        grad_k = df(X_old)

        p_k=np.linalg.solve(Hess_k, -grad_k)

        alpha=backtracking(f, df, X_old, p_k)

        X_new = X_old + alpha * p_k

        if np.linalg.norm(X_new-X_old) < tol_x:
            exit_flag = 'tol_x'
            break

        current_grad_norm=np.linalg.norm(df(X_new))

        X_old = X_new
        count+=1
        fun_history[count] = X_new

    
    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    if count<maxit: # troncamento
        fun_history = fun_history[:count+1]

    return X_old, count, fun_history, exit_flag

## 2.4 Metodo gradiente stocastico

In [27]:
def SGD (df_completo, df_stocastico, X_old, n_samples, max_steps, tol_f, tol_x, k, alpha):

    dim = len(X_old)
    S_k = np.arange(n_samples)

    fun_history = np.zeros((max_steps + 1, dim))
    fun_history[0,]=X_old

    exit_flag = 'maxit'
    epoch_count=0
    total_steps=0

    decay_rate = 0.001

    while total_steps < max_steps:

        epoch_count += 1
        np.random.shuffle(S_k)

        # Ciclo dei mini batch
        for i in range (0, n_samples, k):

            if total_steps >= max_steps:
                break

            indices = S_k[i:i+k] # minibatch
            grad_stoc = df_stocastico(X_old, indices)
            p_k = -grad_stoc

            current_alpha = alpha / (1 + decay_rate * total_steps) # Opzionale, man mano che si va avanti si stabilizza

            X_new = X_old + current_alpha * p_k # bisogna fare backtracking?

            # X_new = np.maximum(X_new, 1e-6)

            total_steps += 1
            fun_history[total_steps] = X_new

            if np.linalg.norm(X_new - X_old) < tol_x:
                exit_flag = 'tol_x'
                return X_new, total_steps, fun_history[:total_steps+1], exit_flag, epoch_count

            X_old = X_new
        # Fine  epoca
        
        full_grad_norm=np.linalg.norm(df_completo(X_new))

        if full_grad_norm < tol_f:
            exit_flag = 'tol_f'
            return X_new, total_steps, fun_history[:total_steps+1], exit_flag, epoch_count

    if total_steps<max_steps: # troncamento
        fun_history = fun_history[:total_steps+1]

    return X_old, total_steps, fun_history, exit_flag, epoch_count

## 2.5 Funzione per la generazione dei grafici

In [8]:
def graph_generator (x0, y0, f, path_history_gd, path_history_newton, tmin, titolo):
    X_mesh, Y_mesh = np.meshgrid(x0, y0)
    Z = f([X_mesh, Y_mesh])

    # Memorizzo il percorso delle x e y
    path_x_gd = path_history_gd[:, 0]
    path_y_gd = path_history_gd[:, 1]
    path_x_new = path_history_newton[:, 0]
    path_y_new = path_history_newton[:, 1]

    # Definisco la figura
    fig = plt.figure(figsize=(14, 8))

    # <-- GENERAZIONE GRAFICO 3D -->

    # Aggiungo subplot 3D
    axs1 = fig.add_subplot(1,2,1,projection='3d')

    # Disegno la superficie
    axs1.plot_surface(X_mesh, Y_mesh, Z, cmap='viridis', rstride=1, cstride=1, linewidth=0, antialiased=True, alpha=0.7)

    # Aggiunge il percorso anche nel grafico 3D gradiente
    z_path_gd = f([path_x_gd, path_y_gd])
    axs1.plot(path_x_gd, path_y_gd, z_path_gd, 'r-o', label='Percorso del Gradiente')

    # Aggiunge il percorso anche nel grafico 3D newton
    z_path_newton = f([path_x_new, path_y_new])
    axs1.plot(path_x_new, path_y_new, z_path_newton, 'b-o', label='Percorso di Newton')

    # Evidenzia il punto iniziale e finale gradiente
    axs1.plot(path_x_gd[0], path_y_gd[0], 'go', label=f'Start Gradiente: ({path_x_gd[0]}, {path_y_gd[0]})') # punto iniziale
    axs1.plot(path_x_gd[-1], path_y_gd[-1], 'rx', label=f'End: ({path_x_gd[-1]:.2f}, {path_y_gd[-1]:.2f})', markersize=16) # punto finale

    # Evidenzia il punto iniziale e finale newton
    axs1.plot(path_x_new[0], path_y_new[0], 'go', label=f'Start Newton: ({path_x_new[0]}, {path_y_new[0]})') # punto iniziale
    axs1.plot(path_x_new[-1], path_y_new[-1], 'rx', label=f'End: ({path_x_new[-1]:.2f}, {path_y_new[-1]:.2f})', markersize=16) # punto finale

    axs1.plot(tmin[0], tmin[1], 'b*', markersize=15, label=f"Minimo Teorico ({tmin[0]}, {tmin[1]})") # Minimo teorico

    # Aggiungo label e titolo
    axs1.set_title(f"{titolo}")
    axs1.set_xlabel("x")
    axs1.set_ylabel("y")
    axs1.set_zlabel("z")
    axs1.legend()
    axs1.view_init(elev=30, azim=135) # Imposta un angolo di visuale

    # <--- GENERAZIONE GRAFICO 2D --->

    # Definisco il grafico 2D
    axs0 = fig.add_subplot(1,2,2)

    # Disegna le curve di livello
    c1 = axs0.contour(X_mesh, Y_mesh, Z, levels=50, cmap='viridis', alpha=0.7)
    fig.colorbar(c1, label='Valore di $f(x,y)$')

    # Disegna il percorso
    axs0.plot(path_x_gd, path_y_gd, 'r-o', label='Percorso del Gradiente')
    axs0.plot(path_x_new, path_y_new, 'b-o', label='Percorso di Newton')

    # Evidenzia il punto iniziale e finale gradiente
    axs0.plot(path_x_gd[0], path_y_gd[0], 'go', label=f'Start: ({path_x_gd[0]}, {path_y_gd[0]})') # punto iniziale
    axs0.plot(path_x_gd[-1], path_y_gd[-1], 'rx', label=f'End: ({path_x_gd[-1]:.2f}, {path_y_gd[-1]:.2f})', markersize=16) # punto finale
    
    # Evidenzia il punto iniziale e finale newton
    axs0.plot(path_x_new[0], path_y_new[0], 'go', label=f'Start: ({path_x_new[0]}, {path_y_new[0]})') # punto iniziale
    axs0.plot(path_x_new[-1], path_y_new[-1], 'bx', label=f'End: ({path_x_new[-1]:.2f}, {path_y_new[-1]:.2f})', markersize=16) # punto finale

    axs0.plot(tmin[0], tmin[1], 'y*', markersize=15, label=f"Minimo Teorico ({tmin[0]}, {tmin[1]})") # Minimo teorico
    # Etichette e titoli
    axs0.set_title(f"{titolo}")
    axs0.set_xlabel("x")
    axs0.set_ylabel("y")
    axs0.legend()
    axs0.set_aspect('equal', adjustable='box') # "Grafo è "quadrato"

    plt.tight_layout()
    plt.show()

### 2.6 Funzione per gli output

In [9]:
def get_solutions(f, df, h_f, x1, x2, maxit, tol_grad, tol_step):
    print("--- Metodo del Gradiente (GD) ---")
    sol_gd, iter_gd, path_gd, exit_gd = GD(f, df, x1, maxit, tol_grad, tol_step)
    print(f"Soluzione: {sol_gd}")
    print(f"Numero iterazioni: {iter_gd}")
    print(f"Condizione di uscita: {exit_gd}")

    print("--- Metodo di Newton ---")
    sol_newton, iter_newton, path_newton, exit_newton = Newton(f, df, h_f, x2, maxit, tol_grad, tol_step)
    print(f"Soluzione: {sol_newton}")
    print(f"Numero iterazioni: {iter_newton}")
    print(f"Condizione di uscita: {exit_newton}")

    return path_gd, path_newton

def get_SGD_solutions(f, sdf, x, n, maxit, tol_grad, tol_step, batch_k, learning_rate):
    print("--- Metodo del Gradiente Stocastico (SGD) ---")
    sol_sgd, iter_sgd, path_sgd, exit_sgd, epoch = SGD(f, sdf, x, n, maxit, tol_grad, tol_step, batch_k, learning_rate)
    print(f"Soluzione: {sol_sgd}")
    print(f"Numero iterazioni: {iter_sgd}")
    print(f"Condizione di uscita: {exit_sgd}")
    print(f"Epoche fatte: {epoch}")

    return path_sgd

In [10]:
def error_graph(f, df, gd, newton, titolo, sgd=None): 
    
    errori_gd = [f(x_k) for x_k in gd]
    errori_newton = [f(x_k) for x_k in newton]

    norma_grad_gd = [np.linalg.norm(df(x_k)) for x_k in gd]
    norma_grad_newton = [np.linalg.norm(df(x_k)) for x_k in newton]

    fig, axs = plt.subplots(2, 2, figsize=(16, 10)) 

    ax = axs[0, 0]
    ax.plot(errori_gd, label="Gradient Descent (GD)", marker='.', linestyle='-')
    ax.plot(errori_newton, label="Newton Smorzato", marker='.', linestyle='--')
    ax.set_title("Valore Funzione (GD vs Newton)")
    ax.set_ylabel("Valore Funzione $f(x_k)$ (log scale)")
    ax.set_xlabel("Iterazione (k)")
    ax.set_yscale('log')
    ax.legend()
    ax.grid(which="both", linestyle='--', linewidth=0.5)

    ax = axs[1, 0]
    ax.plot(norma_grad_gd, label="Gradient Descent (GD)", marker='.', linestyle='-')
    ax.plot(norma_grad_newton, label="Newton Smorzato", marker='.', linestyle='--')
    ax.set_title("Norma Gradiente (GD vs Newton)")
    ax.set_ylabel(r"Norma Gradiente $\|\nabla f(x_k)\|$ (log scale)")
    ax.set_xlabel("Iterazione (k)")
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, which="both", linestyle='--', linewidth=0.5)


    if sgd is not None:
        errori_sgd = [f(x_k) for x_k in sgd]
        norma_grad_sgd = [np.linalg.norm(df(x_k)) for x_k in sgd]

        ax = axs[0, 1]
        ax.plot(errori_sgd, label="Stochastic GD", color='green', alpha=0.8)
        ax.set_title("Valore Funzione (SGD)")
        ax.set_ylabel("Valore Funzione $f(x_k)$ (log scale)")
        ax.set_xlabel("Iterazione (k)")
        ax.set_yscale('log')
        ax.legend()
        ax.grid(True, which="both", linestyle='--', linewidth=0.5)

        ax = axs[1, 1]
        ax.plot(norma_grad_sgd, label="Stochastic GD", color='green', alpha=0.8)
        ax.set_title("Norma Gradiente (SGD)")
        ax.set_ylabel(r"Norma Gradiente $\|\nabla f(x_k)\|$ (log scale)")
        ax.set_xlabel("Iterazione (k)")
        ax.set_yscale('log')
        ax.legend()
        ax.grid(which="both", linestyle='--', linewidth=0.5)
    
    else:
        axs[0, 1].axis('off')
        axs[1, 1].axis('off')

    fig.suptitle(f"Analisi Convergenza per {titolo}", fontsize=18)
    
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    plt.show()

--- 
## 3 Funzione Quadratica: $f(x, y) = (x-5)^2+(y-2)^2$

In [11]:
def f1(X):
    x = X[0]
    y = X[1]
    return (x-5)**2+(y-2)**2

def df1(X):
    x = X[0]
    y = X[1]

    df_dx = 2*(x-5) 
    df_dy = 2*(y-2)
    
    return np.array([df_dx, df_dy])

def hess_f1(X):
    HESS=np.zeros((2,2))

    HESS[0][0] = 2
    HESS[0][1] = 0
    HESS[1][0] = 0
    HESS[1][1] = 2
    return HESS    

x_range1=np.linspace(-2,12,200)
y_range1=np.linspace(-2,8,200)

maxit = 300
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto1_gd=np.array([0.0,0.0])
punto1_new=np.array([1.0,1.0])
tmin1 = [5.0, 2.0]
titolo1 = r"$f(x, y) = (x-5)^2+(y-2)^2$"

path_gd_f1, path_new_f1 = get_solutions(f1, df1, hess_f1, punto1_gd, punto1_new, maxit, tolleranza_f, tolleranza_x)

--- Metodo del Gradiente (GD) ---
Soluzione: [5. 2.]
Numero iterazioni: 1
Condizione di uscita: tol_f
--- Metodo di Newton ---
Soluzione: [5. 2.]
Numero iterazioni: 1
Condizione di uscita: tol_f


In [13]:
graph_generator(x_range1, y_range1, f1, path_gd_f1, path_new_f1, tmin1, titolo1)

--- 
## 4. Funzione di Rosenbrock: $f(x, y) = (1-x)^2+100(y-x^2)^2$

In [ ]:
def f2(X):
    x = X[0]
    y = X[1]

    return (1-x)**2+100*(y-x**2)**2

def df2(X):
    x = X[0]
    y = X[1]

    df_dx = -2*(1-x) - 400*(y-x**2)*x
    df_dy = 200*(y-x**2)
    
    return np.array([df_dx, df_dy])

def hess_f2(X):
    HESS=np.zeros((2,2)) # Crea una matrice 2x2
    x = X[0]
    y = X[1]

    HESS[0][0] = 2 - 400*y + 1200*x**2 
    HESS[0][1] = -400*x 
    HESS[1][0] = HESS[0][1] # Per teorema di Sxhwarz
    HESS[1][1] = 200 
    return HESS
    

x_range2=np.linspace(-2,2,200)
y_range2=np.linspace(-1,3,200)

maxit = 5000
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto2_gd=np.array([0.0,0.0])
punto2_new=np.array([0.5,0.5])
tmin2 = [1.0,1.0]
titolo2 = r"$f(x, y) = (1-x)^2+100(y-x^2)^2$"

path_gd_r, path_newton_r = get_solutions(f2, df2, hess_f2, punto2_gd, punto2_new, maxit, tolleranza_f, tolleranza_x)

In [ ]:
graph_generator(x_range2, y_range2, f2, path_gd_r, path_newton_r, tmin2, titolo2)
error_graph(f2, df2, path_gd_r, path_newton_r, titolo2)

---
## 5 Funzione multimodale: $f(x,y) = x^4-x^2+y^2$

In [ ]:
def f3(X):
    x = X[0]
    y = X[1]

    return x**4-x**2+y**2

def df3(X):
    x = X[0]
    y = X[1]

    df_dx = 4*x**3-2*x
    df_dy = 2*y
    
    return np.array([df_dx, df_dy])

def hess_f3(X):
    HESS=np.zeros((2,2))
    x = X[0]
    y = X[1]

    HESS[0][0] = 12*x**2-2 # Derivata seconda rispetto a x
    HESS[0][1] = 0 # Derivata mista
    HESS[1][0] = HESS[0][1] # Per teorema di Sxhwarz, la derivata mista è uguale
    HESS[1][1] = 2 # Derivata seconda rispetto a y
    return HESS
    

x_range3=np.linspace(-1.5,1.5,200)
y_range3=np.linspace(-1.5,1.5,200)

maxit = 5000
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto3_neg=np.array([-1.0,-1.0]) 
punto3_pos=np.array([1.0,1.0]) 
punto3_sad=np.array([0.0,5.0]) 
tmin3 = [0.0, 0.0]
tmin3_alt = [-np.sqrt(1/2), 0.0]
titolo3 = r"$f(x,y) = x^4-x^2+y^2$"

path_gd_m, path_new_m = get_solutions(f3, df3, hess_f3, punto3_pos, punto3_neg, maxit, tolleranza_f, tolleranza_x)

In [ ]:
# Gradiente
graph_generator(x_range3, y_range3, f3, path_gd_m, path_new_m, tmin3, titolo3)
error_graph(f3, df3, path_gd_m, path_new_m, titolo3)

---
# 6. Minimi Quadrati Lineari: $f(x) = \frac12||Ax-b||_2^2$ con $A \in \R^{n \times n}$

In [25]:
def f4(x, A, b):
    e = A @ x -b
    norm_sq = np.dot(e, e)
    return 1/2 * norm_sq

def df4(X, A, b):
    return A.T @ (A @ X - b)

def hess_f4(A):
    return A.T @ A

def sdf4(x, indices, A, b):
    A_batch= A[indices, :]
    b_batch= b[indices]
    
    return A_batch.T @ (A_batch @ x - b_batch)

n = 50 
A = np.random.rand(n,n) 
b = A @ np.ones(n) 
X_start_1 = np.zeros(n) 
X_start_2 = np.random.randn(n)

title_f4 = r"$f(x) = \frac{1}{2}\|Ax-b\|_2^2 \text{ con } A \in \mathbb{R}^{n \times n}$"
maxit = 2000
maxit_SGD = 20000
tol_grad = 1e-3
tol_step = 1e-6
batch_k = 5
learning_rate = 0.001

f_min_sq = lambda x: f4(x, A, b) # Equivalente a scrivere def f_min_sq(x): return f4(x, A, b)
df_min_sq = lambda x: df4(x, A, b) 
h_min_sq = lambda x: hess_f4(A)
sdf_min_sq = lambda x, idx: sdf4(x, idx, A, b)

path_gd_f4, path_newton_f4 = get_solutions(f_min_sq, df_min_sq, h_min_sq, X_start_1, X_start_2, maxit, tol_grad, tol_step)
path_sgd_f4 = get_SGD_solutions(df_min_sq, sdf_min_sq, X_start_1, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)

--- Metodo del Gradiente (GD) ---
Soluzione: [0.99625704 1.00355389 1.00625273 0.99168709 1.00979348 1.00182711
 1.01014478 0.99395728 1.00008295 0.99502734 0.9858794  0.99914372
 1.01017622 1.00076338 0.98224393 0.9993252  0.99016675 0.98258641
 1.01481681 1.00137066 1.00096128 1.00664415 1.00600673 1.01206293
 0.99271109 0.99909923 1.00198527 0.99908576 1.00704416 0.99822909
 1.01263554 1.003518   1.00757365 1.00189956 0.99148729 1.01555899
 0.99079325 0.99727536 1.01353544 1.00038796 0.98921114 1.01617054
 0.99864073 0.9820591  0.98926317 0.98452137 1.00445715 0.99545007
 1.00538258 0.99424881]
Numero iterazioni: 2000
Condizione di uscita: maxit
--- Metodo di Newton ---
Soluzione: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Numero iterazioni: 1
Condizione di uscita: tol_f
--- Metodo del Gradiente Stocastico (SGD) ---
Soluzione: [0.9818633  1.0268749  1.01615164 0.97604503 1.

In [32]:
error_graph(f_min_sq, df_min_sq, path_gd_f4, path_newton_f4, title_f4, path_sgd_f4)

Notiamo che newton è velocissimo. Questo perchè in funzioni quadrate, salta direttamente in quel punto.
Il grafico del gradiente è rumoroso, perchè nella iterazione k fa un passo fuori rotta, il grafico sale, per poi essere ritornato nella iterazione k+1. 

---
# 7 Minimi quadrati con regolarizzazione: $f(x) = \frac{1}{2}||Ax-b||^2_2 + \lambda ||x||^2_2$, con $\lambda \in [0,1]$

In [30]:
# Definisco la funzione
def f5(x, A, b, lambda_val):
    e = A @ x -b
    errore_ls = 1/2 * np.dot(e, e)
    errore_reg = lambda_val*np.dot(x,x)
    return errore_ls + errore_reg

# Questa calcola il GRADIENTE COMPLETO (Batch)
def df5(x, A, b, lambda_val):
    grad_ls = A.T @ (A @ x - b)
    grad_reg = 2*lambda_val*x
    return grad_ls +  grad_reg

# Hessiana
def hess_f5(A, lambda_val):
    return A.T @ A + 2 * lambda_val * np.identity(n)

# Questa calcola il GRADIENTE STOCASTICO (Mini-Batch)
def sdf5(x, indices, A, b, lambda_val):
    A_batch= A[indices, :]
    b_batch= b[indices]

    grad_ls_stoc = A_batch.T @ (A_batch @ x - b_batch)
    grad_reg = 2 * lambda_val * x
    
    return grad_ls_stoc + grad_reg

n = 50 # siamo in R^50
A = np.random.rand(n,n) # A è una matrice n x n con valori casuali
X_true = np.ones(n)
b = A @ X_true + 0.1 * np.random.randn(n) # rendiamo b in modo che b= A(vettori 1)+rumore
lambda_val = 0.1 

titolo_f5 = r"$f(x) = \frac{1}{2}\|Ax-b\|_2^2 + \lambda \|x\|_2^2 \text{ con } \lambda \in [0,1]$"
X_start1 = np.zeros(n)      # Punto di partenza
X_start2 = np.ones(n)      # Punto di partenza
maxit = 2000
maxit_SGD = 20000         # L'SGD richiede molti più passi del GD
batch_k = 5                # Dimensione del mini-batch
learning_rate = 0.001      # Alpha (deve essere molto piccolo)
# NOTA: Una epoca è fatta ad ogni 10 iterazioni, perchè n/batch_k = 50 / 5 = 10

f_min_sqr = lambda x: f5(x, A, b, lambda_val)
df_min_sqr = lambda x: df5(x, A, b, lambda_val)
h_min_sqr = lambda x: hess_f5(A, lambda_val)
sdf_min_sqr = lambda x, idx: sdf5(x, idx, A, b, lambda_val)

path_gd_f5, path_newton_f5 = get_solutions(f_min_sqr, df_min_sqr, h_min_sqr, X_start1, X_start2, maxit, tol_grad, tol_step)
path_sgd_f5 = get_SGD_solutions(df_min_sqr, sdf_min_sqr, X_start1, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)

--- Metodo del Gradiente (GD) ---
Soluzione: [1.09421764 1.02032915 0.98984194 0.97438675 1.00971359 1.04545569
 1.09582169 0.91199246 0.96950708 0.99887427 1.01150332 1.01955691
 1.05798793 0.9596553  0.91455621 1.07600929 1.10620473 1.02498181
 0.91155935 1.12399844 1.01124341 1.03585899 0.95155458 0.94081766
 0.89462379 0.91183107 0.96745449 1.12894876 0.92117308 0.9785689
 1.03305295 0.88022418 0.88126336 0.92863784 1.04620917 1.10802152
 1.01808081 0.99435567 0.96735169 1.10158136 0.99500153 0.95350467
 0.9991381  1.10595702 0.9735715  1.02307288 0.93558356 0.99619668
 1.01273769 0.99838875]
Numero iterazioni: 2000
Condizione di uscita: maxit
--- Metodo di Newton ---
Soluzione: [1.0960548  1.01941702 0.98959346 0.97579331 1.00896934 1.04624707
 1.09608428 0.91439047 0.96950788 0.99787836 1.01136811 1.01925969
 1.06034389 0.95911814 0.91310555 1.07524609 1.10709169 1.0262
 0.91162969 1.1233173  1.01330904 1.03546912 0.95132799 0.94116699
 0.89504282 0.91072567 0.96742154 1.12855332

In [31]:
error_graph(f_min_sqr, df_min_sqr, path_gd_f5, path_newton_f5, titolo_f5, path_sgd_f5)

---
# 8 $f(x) = \sum \limits_{i=1}^n (x_i-i)^2-\sum \limits_{i=1}^n \ln(x_i)$, con $x_i>0$

In [28]:
def f6 (x, idx):
    if np.any(x <= 0):
        return np.inf # ritorna infinito se x non è nel dominio (ln(x) con x<=0)

    sum1 = np.sum((x-idx)**2)
    sum2 = np.sum(np.log(x))
    return sum1 - sum2
    
def df6 (x, idx):
    return 2 * (x - idx) - 1/x

def hess_f6 (x):
    HESS = 2+(1/(x**2))
    return np.diag(HESS)

def sdf6(x, mini_batch, n, idx):

    x_batch = x[mini_batch]
    idx_batch = idx[mini_batch]
    grad_stoc = np.zeros(n)
    
    grad_batch_components = 2 * (x_batch - idx_batch) - (1 / x_batch)
    
    grad_stoc[mini_batch] = grad_batch_components
    
    return grad_stoc

title_f6 = r"$f(x) = \sum_{i=1}^n (x_i-i)^2 - \sum_{i=1}^n \ln(x_i)$"
n = 50
maxit_SGD = 10000
batch_k = 10
idx = np.arange(1, n+1)
X_start1 = np.ones(n)
X_start2 = np.random.rand(n) + 1

f_lambda = lambda x: f6(x, idx)
df_lambda = lambda x: df6(x, idx)
h_lambda = lambda x: hess_f6(x)
sdf_lambda = lambda x, k: sdf6(x, k, n, idx)

path_gd_f6, path_newton_f6 = get_solutions(f_lambda, df_lambda, h_lambda, X_start1, X_start2, maxit, tol_grad, tol_step)
path_sgd_f6 = get_SGD_solutions(df_lambda, sdf_lambda, X_start2, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)

--- Metodo del Gradiente (GD) ---
Soluzione: [ 1.36585366  2.22474227  3.1583123   4.12132034  5.09807621  6.082207
  7.07071421  8.0620192   9.05521679 10.04975247 11.04526825 12.04152299
 13.03834842 14.03562364 15.03325959 16.0311892  17.02936105 18.02773504
 19.02627944 20.02496883 21.02378259 22.02270384 23.02171862 24.02081528
 25.01998403 26.01921657 27.01850583 28.01784577 29.01723114 30.01665742
 31.01612065 32.01561738 33.01514456 34.01469953 35.01427989 36.01388353
 37.01350858 38.01315334 39.0128163  40.0124961  41.0121915  42.01190139
 43.01162476 44.0113607  45.01110837 46.010867   47.01063589 48.01041441
 49.01020196 50.009998  ]
Numero iterazioni: 6
Condizione di uscita: tol_f
--- Metodo di Newton ---
Soluzione: [ 1.3660254   2.22474487  3.15831235  4.12132032  5.0980762   6.082207
  7.07071421  8.0620192   9.05521678 10.04975247 11.04526825 12.04152298
 13.03834842 14.03562364 15.03325959 16.0311892  17.02936105 18.02773504
 19.02627944 20.02496883 21.02378259 22.02270

In [29]:
error_graph(f_lambda, df_lambda, path_gd_f6, path_newton_f6, title_f6, path_sgd_f6)

---
# 9 $f(x) = \sum \limits_{i=1}^n (x_i-b_i)^2+c\sum \limits_{i=1}^n \ln(x_i)$

In [21]:
def f7(x,c,b):

    sum1 = np.sum((x-b)**2)
    sum2 = c*np.sum(np.log(x))

    return sum1+sum2

def df7(x,c,b):
    return 2*(x-b)+c*(1/x)

def hess_f7(x,c):
    HESS =2-c*(1/(x**2))
    return np.diag(HESS)

def sdf7(x, c, b, n, mini_batch):
    x_batch = x[mini_batch]
    b_batch = b[mini_batch]
    grad_stoc = np.zeros(n)
    
    grad_batch_components = 2 * (x_batch - b_batch) + c*(1 / x_batch)
    grad_stoc[mini_batch] = grad_batch_components
    
    return grad_stoc

title_f7 = r"$f(x) = \sum_{i=1}^n (x_i-b_i)^2+c\sum_{i=1}^n \ln(x_i)$"
b = np.arange(1, n+1)
X_start1 = np.ones(n)
X_start2 = np.random.rand(n)+0.1
c= -2.0 # DEVE essere negativo

f_c = lambda x: f7(x, c, b)
df_c = lambda x: df7(x, c, b)
h_c = lambda x: hess_f7(x, c)
sdf_c = lambda x, k: sdf7(x, c, b, n, k)

path_gd_f7, path_newton_f7 = get_solutions(f_c, df_c, h_c, X_start1, X_start2, maxit, tol_grad, tol_step)
path_sgd_f7 = get_SGD_solutions(f_c, sdf_c, X_start1, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)

--- Metodo del Gradiente (GD) ---
Soluzione: [ 1.61818182  2.41421393  3.30277564  4.23606798  5.1925824   6.16227766
  7.14005494  8.12310563  9.10977223 10.09901951 11.09016994 12.08276253
 13.07647322 14.07106781 15.06637298 16.06225775 17.05862138 18.05538514
 19.05248659 20.04987562 21.04751155 22.04536102 23.04339638 24.04159458
 25.0399362  26.03840481 27.03698637 28.03566885 29.03444185 30.03329638
 31.03222457 32.03121954 33.03027525 34.02938637 35.02854814 36.02775638
 37.02700731 38.02629759 39.02562419 40.02498439 41.02437575 42.02379604
 43.02324325 44.02271555 45.02221126 46.02172887 47.02126697 48.0208243
 49.02039967 50.01999201]
Numero iterazioni: 9
Condizione di uscita: tol_f
--- Metodo di Newton ---
Soluzione: [ 1.61803399  2.41421356  3.30277564  4.23606798  5.1925824   6.16227766
  7.14005494  8.12310406  9.10977223 10.09901951 11.09016994 12.08276253
 13.07647322 14.07106781 15.06637298 16.06225775 17.05862138 18.05538514
 19.05248659 20.04987562 21.04751155 22.04

In [22]:
error_graph(f_c, df_c, path_gd_f7, path_newton_f7, title_f7, path_sgd_f7)